In [1]:
import pandas as pd
import sys
import io
import time
from itertools import combinations
from gurobipy import *
from clean_data_cor import *

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
def read_data(path_dem:str, path_preco:str, path_rota:str):

    demanda = pd.read_csv(path_dem)
    preco = pd.read_csv(path_preco)
    rota0 = pd.read_csv(path_rota)
    rota1 = eval(rota0['Route'][0])

    return demanda, preco, rota1

def create_sets(demanda, preco_, rota1, perio=0):

    if perio != 0:
        periodo_lim = sorted(demanda['DBD'].unique().tolist())[:perio]
        demanda = demanda[demanda['DBD'].isin(periodo_lim)]

    rota = [0] + rota1
    
    I, J, OD, V,  T, stations, VK, P, d = clean_data(demanda, preco_)

    # AD = [(i,j) for i,j in OD if rota1.index(j) == rota1.index(i)+1]
    NAD = [(i,j) for i,j in OD if rota1.index(j) != rota1.index(i)+1]

    I = [i for i in rota if i in I]
    I2 = [0] + I# tambem se toma para el para tudo da restriccion de fluxo
    J = [i for i in rota if i in J]
    n = len(rota)-1

    BR = {}
    for i,j in NAD:
        listTemp = list(combinations(rota1[rota1.index(i):rota1.index(j)+1], 2))
        listTemp =  {(ii,jj) for ii,jj in listTemp if (ii,jj) in OD and (ii, jj) != (i,j)} #rota1.index(jj) == rota1.index(ii)+1 and 
        BR[(i,j)] = listTemp

    # indices
    index0 = [(i,j,v,k,t) for i in I2 for j in J for v in V for k in VK[v] for t in T if rota.index(i) <  rota.index(j) ]#and (i,j) in OD]
    index1 = [(i,j,v,k,t) for i in I for j in J for v in V for k in VK[v] for t in T if rota.index(i) <  rota.index(j) ]#and (i,j) in OD]
    index = [(i,j,v,k,t) for i,j in BR.keys() for v in V for k in VK[v] for t in T if rota.index(i) <  rota.index(j) ]#and (i,j) in OD]

    return I, I2, J, OD, NAD, V,  T, stations, VK, P, d, n, BR, I2, index0, index, index1

def signum(x):
    return 0 if x<= 0 else 1

def create_model(I, I2, J, V, VK, T, OD, NAD, BR, P, Q, d, index0, index, index1, rota):
    model = Model("Modelo 1.1.1")

    # variables de decicion
    X = model.addVars(index0, vtype=GRB.INTEGER , name="X")
    Y = model.addVars(index1, vtype=GRB.INTEGER , name="Y")
    A = model.addVars(I2, vtype=GRB.INTEGER , name="A")
    BNA = model.addVars(index, vtype=GRB.BINARY , name="BNA")


    # funcion objetivo
    model.setObjective(
        quicksum(P[(i,j,v,k)]*X[(i,j,v,k,t)] for i,j in OD for v in V for k in VK[v] for t in T), 
        sense = GRB.MAXIMIZE
    )

    # restricciones
    for i in I:
        
        model.addConstr(
            A[i] == A[rota[rota.index(i)-1]] - quicksum(X[rota[rota.index(i)-1],j,v,k,t] for j in J for v in V for k in VK[v] for t in T if (rota[rota.index(i)-1] != j) and (rota[rota.index(i)-1],j) in OD) + quicksum(X[j,i,v,k,t] for j in I for v in V for k in VK[v] for t in T if rota.index(j) < rota.index(i) and (j,i) in OD),
            name=f"Dispo_{i}"
        )

        model.addConstr(
            quicksum((X[i,j,v,k,t]) for j in J for v in V for k in VK[v] for t in T if i != j and (i,j) in OD) <= A[i],
            name=f"Cap_{i}"
        )

        model.addConstr(
            quicksum(Y[i,j,v,VK[v][0],t] for j in J for v in V for t in T if i != j and (i,j) in OD) <= Q, 
            name=f"AuthoCap_{i}"
        )



    for i,j in OD:
        for v in V:
            for k in VK[v]:
                for t in T:
                    model.addConstr(
                        X[i,j,v,k,t] <= d[i,j,v,k,t],
                        name = f"Assig_({i},{j},{v},{k},{t})"
                    )

                    if k != VK[v][-1]:
                        pos_k = VK[v].index(k)
                        model.addConstr(
                            Y[i,j,v,k,t] >= Y[i,j,v,VK[v][pos_k+1],t], 
                            name=f"Classe_({i},{j},{v},{k},{t})"
                        )

                        model.addConstr(
                            Y[i,j,v,k,t] >=  X[i,j,v,k,t] + Y[i,j,v,VK[v][pos_k+1],t],
                            name=f"Autho_({i},{j},{v},{k},{t})"
                        )
                    else:
                        model.addConstr(
                            Y[i,j,v,k,t] >=  X[i,j,v,k,t],
                            name=f"Autho_({i},{j},{v},{k},{t})"
                        )

                    

    # [start] --- Restricciones de Capitalismo
    for o,d_ in NAD:

        for v in V:
            for k in VK[v]:
                for t in T:

                    model.addConstr(
                        BNA[o,d_,v,k,t] <= Y[o,d_,v,k,t],
                        # name = f"activ_bin_autho_low_({o},{d_},{v},{k},{t})"
                    )

                    model.addConstr(
                        Y[o,d_,v,k,t] <= Q*BNA[o,d_,v,k,t],
                        # name = f"activ_bin_autho_top_({o},{d_},{v},{k},{t})"
                    )

        for i,j in BR[o,d_]:
            for v in V:
                for k in VK[v]:
                    for t in T:

                        model.addConstr(
                            Y[i,j,v,k,t] <= Q*BNA[o,d_,v,k,t],
                            # name = f"pru1({o},{d_},{i},{j},{v},{k},{t})"
                        )

                        model.addConstr(
                            BNA[o,d_,v,k,t] <= Y[i,j,v,k,t],
                            # name = f"pru2({o},{d_},{i},{j},{v},{k},{t})"
                        )


    # [end] --- Restricciones de Capitalismo


    for j in J:
        for v in V:
            for k in VK[v]:
                for t in T:
                    model.addConstr(
                        X[0,j,v,k,t] == 0,
                        name = f"Assig_({0},{j},{v},{k},{t})"
                    )

                    # model.addConstr(
                    #     Y[0,j,v,k,t] == 0,
                    #     name = f"Autho_({0},{j},{v},{k},{t})"
                    # )

    model.addConstr(
        A[0] == Q,
        name = f"Cap_0"
    )

    return model, A, X, Y, BNA

def save_solution(model, OD, V, VK, T, BR, P, d, X, Y, BNA, perio, instance):
    print('Valor da função objetivo: ', str(model.ObjVal) )
    print('')
    lista = []
    for i,j in OD:
        for v in V:
            for k in VK[v]:
                for t in T:
                    if (i,j) in BR.keys():
                        lista.append([i+'-'+j,i,j,v, k, t, P[i,j,v,k], d[i,j,v,k,t], X[i,j,v,k,t].X, Y[i,j,v,k,t].X , BNA[i,j,v,k,t].X ])
                    else:
                        lista.append([i+'-'+j,i,j,v, k, t, P[i,j,v,k], d[i,j,v,k,t], X[i,j,v,k,t].X, Y[i,j,v,k,t].X , -1 ])

    a = pd.DataFrame(lista, columns=['o-d',"Origen","Destino",'Vagon','classe','Periodo','Preco','Demanda','Assignments','Authorizations','Binaria'])
    
    a.to_excel('modelo1_'+instance+'_t'+str(perio)+'.xlsx', index=False)

In [3]:
# parametros
instances = ['instancia3']#['instancia1', 'instancia2','instancia3']#
Qs = [565]#[561, 565, 565]#
num_periodos = [5]#,10,20,40,50,60]

In [4]:
start_time_exp = time.time()
for instance in instances:
    
    path_dem = '/home/wilmer/Documentos/Codes/tesis/instancias/'+instance+'/demanda.csv'
    path_preco = '/home/wilmer/Documentos/Codes/tesis/instancias/'+instance+'/preco.csv'
    path_rota = '/home/wilmer/Documentos/Codes/tesis/instancias/'+instance+'/rota.csv'
    
    Q = Qs[instances.index(instance)]

    # chamar dados
    demanda, preco_, rota1 = read_data(path_dem, path_preco, path_rota)

    for num_periodo in num_periodos:
        # Criar Conjuntos
        I, I2, J, OD, NAD, V,  T, stations, VK, P, d, n, BR, rota, index0, index, index1 = create_sets(demanda, preco_, rota1, num_periodo)

        # criar modelo
        start_time = time.time()
        model, A, X, Y, BNA = create_model(I, I2, J, V, VK, T, OD, NAD, BR, P, Q, d, index0, index, index1, rota)
        end_time = time.time()
        tempo = end_time - start_time

        # # Redirigir la salida estándar a un archivo
        # old_stdout = sys.stdout
        # new_stdout = io.StringIO()
        # sys.stdout = new_stdout

        # Optimizar o modelo
        model.optimize()

        # Salvar solucao
        save_solution(model, OD, V, VK, T, BR, P, d, X, Y, BNA, num_periodo, instance)
        print('Tempo para criar o modelo: ', tempo)

        # # Volver a la salida estándar
        # sys.stdout = old_stdout

        # # Obtener la salida de la optimización
        # output = new_stdout.getvalue()

        # # Guardar la salida en un archivo de texto
        # with open('modelo1_'+instance+'_t'+str(num_periodo)+'.txt', 'w') as f:
        #     f.write(output)

end_time_exp = time.time()
all_time = end_time_exp - start_time_exp


Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-11


/home/wilmer/Documentos/Codes/tesis/gurobi/clean_data_cor.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  records_to_correct['clave_completa'] = records_to_correct['Origin'].astype(str) + '_' + records_to_correct['Destination'].astype(str) + '_' + records_to_correct['Vagon'].astype(str) + '_' + records_to_correct['Class'].astype(str) + '_' + records_to_correct['DBD'].astype(str)


AttributeError: Index out of range for attribute 'X'

In [ ]:
# # chamar dados
# demanda, preco_, rota1 = read_data(path_dem, path_preco, path_rota)

# # Criar Conjuntos
# I, I2, J, OD, NAD, V,  T, stations, VK, P, d, n, BR, rota, index0, index = crate_sets(demanda, preco_, rota1, num_periodo)

# # criar modelo
# model, A, X, Y, BNA = create_model(I, I2, J, V, VK, T, OD, NAD, BR, P, Q, d, index0, index, rota)

# # Redirigir la salida estándar a un archivo
# old_stdout = sys.stdout
# new_stdout = io.StringIO()
# sys.stdout = new_stdout

# # Optimizar o modelo
# model.optimize()

# # Salcar solucao
# save_solution(model, OD, V, VK, T, BR, P, d, X, Y, BNA, num_periodo)

# # Volver a la salida estándar
# sys.stdout = old_stdout

# # Obtener la salida de la optimización
# output = new_stdout.getvalue()

# # Guardar la salida en un archivo de texto
# with open('modelo1_',instance+'_t'+str(num_periodo)+'.txt', 'w') as f:
#     f.write(output)

In [ ]:
# import random 
# rota1 = ['uno','dos','tres', 'cuatro', 'cinco']
# rota = [0] + rota1

# OD = [('uno','dos'),
#            ('uno','tres'),
#            ('uno','cuatro'),
#            ('uno','cinco'),
#            ('dos','tres'),
#            ('dos','cuatro'),
#            ('dos','cinco'),
#            ('tres','cuatro'),
#            ('tres','cinco'),
#            ('cuatro','cinco'),]

# AD = [(i,j) for i,j in OD if rota1.index(j) == rota1.index(i)+1]
# NAD = [(i,j) for i,j in OD if rota1.index(j) != rota1.index(i)+1]

# I = ['uno','dos','tres', 'cuatro']
# I2 = [0] + I

# J = ['dos','tres','cuatro', 'cinco']
# V = ['p','z']
# T = [0,1]
# Q = 700 
# n = len(rota) - 1 
# VK = {'p':[1,5,7],'z':[1,3,7]}

# d = {(i,j,v,k,t): np.random.randint(1,100) for i,j in OD for v in V for k in VK[v] for t in T }

# P = {}
# for i,j in OD:
#     for v in V:
#         aleat = sorted(random.sample(range(50, 2000 + 1), len(VK[v])), reverse=True)
#         for k,valor in enumerate(aleat):
#             P[(i,j,v,VK[v][k])] = valor

# # corrigir instancias
# d = {(i,j,v,k,t): d[(i,j,v,k,t)] if (i,j,v,k,t) in d else 0  for i,j in OD for v in V for k in VK[v] for t in T}
# P = {(i,j,v,k): P[(i,j,v,k)] if (i,j,v,k) in P else 0  for i,j in OD for v in V for k in VK[v] }